In [17]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestRegressor 

from sklearn.metrics import mean_absolute_error 
from catboost import CatBoostRegressor 

from xgboost import XGBRegressor


In [18]:
df_train = pd.read_csv("./data/df_train_sinmodelosunicos.csv") 
df_test = pd.read_csv("./data/df_test_prueba_larga.csv") 

In [20]:
X = df_train.drop(columns=['Price_euros'])
y = df_train['Price_euros']

# Dividir el conjunto de datos en entrenamiento y prueba
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Identificar columnas numéricas y categóricas
num_features = X_train.select_dtypes(include=['int64', 'float64']).columns
cat_features = X_train.select_dtypes(include=['object']).columns

# Preprocesamiento para columnas numéricas
num_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])

# Preprocesamiento para columnas categóricas
cat_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Crear el ColumnTransformer para aplicar el preprocesamiento adecuado a cada columna
preprocessor = ColumnTransformer(
    transformers=[
        ('num', num_transformer, num_features),
        ('cat', cat_transformer, cat_features)
    ])

# Definir diferentes combinaciones de hiperparámetros para probar en RandomForestRegressor
hyperparameters = [{'n_estimators': 139}]
best_mae = float('inf')
best_model = None
best_hyperparameters = None

# Probar diferentes combinaciones de hiperparámetros en RandomForestRegressor
for params in hyperparameters:
    # Crear el pipeline completo con el nuevo hiperparámetro
    pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', RandomForestRegressor(**params, random_state=42))
    ])

    # Entrenar el modelo
    pipeline.fit(X_train, y_train)

    # Evaluar el modelo en el conjunto de prueba
    y_pred = pipeline.predict(X_test)
    mae = mean_absolute_error(y_test, y_pred)

    # Guardar el mejor modelo y sus hiperparámetros
    if mae < best_mae:
        best_mae = mae
        best_model = pipeline
        best_hyperparameters = params

# Definir diferentes combinaciones de hiperparámetros para probar en XGBoost
xgb_hyperparameters = [
                    
                        {'n_estimators': 123, 'learning_rate': 0.12},
                        {'n_estimators': 124, 'learning_rate': 0.12}, 
                        {'n_estimators': 125, 'learning_rate': 0.12}


 

                       ]

# Mejor MAE y modelo para XGBoost
best_xgb_mae = float('inf')
best_xgb_model = None
best_xgb_hyperparameters = None

# Probar diferentes combinaciones de hiperparámetros en XGBoost
for xgb_params in xgb_hyperparameters:
    # Crear el pipeline completo con el nuevo hiperparámetro
    xgb_pipeline = Pipeline(steps=[
        ('preprocessor', preprocessor),
        ('regressor', XGBRegressor(**xgb_params, random_state=42))
    ])

    # Entrenar el modelo
    xgb_pipeline.fit(X_train, y_train)

    # Evaluar el modelo en el conjunto de prueba
    y_xgb_pred = xgb_pipeline.predict(X_test)
    xgb_mae = mean_absolute_error(y_test, y_xgb_pred)

    # Guardar el mejor modelo y sus hiperparámetros
    if xgb_mae < best_xgb_mae:
        best_xgb_mae = xgb_mae
        best_xgb_model = xgb_pipeline
        best_xgb_hyperparameters = xgb_params

# Imprimir el MAE más bajo y los hiperparámetros correspondientes de RandomForestRegressor
print("Mejor MAE de RandomForestRegressor:", best_mae)
print("Mejores hiperparámetros de RandomForestRegressor:", best_hyperparameters)

# Imprimir el MAE más bajo y los hiperparámetros correspondientes de XGBoost
print("Mejor MAE de XGBoost:", best_xgb_mae)
print("Mejores hiperparámetros de XGBoost:", best_xgb_hyperparameters)

# Si el mejor modelo es XGBoost, usarlo para predecir los precios en el conjunto de prueba
if best_xgb_mae < best_mae:
    predicciones = best_xgb_model.predict(df_test)
else:
    predicciones = best_model.predict(df_test)

# Añadir las predicciones al dataframe de prueba
df_test['Price_euros'] = predicciones



Mejor MAE de RandomForestRegressor: 156.25645315565728
Mejores hiperparámetros de RandomForestRegressor: {'n_estimators': 139}
Mejor MAE de XGBoost: 150.33629984422166
Mejores hiperparámetros de XGBoost: {'n_estimators': 124, 'learning_rate': 0.12}


In [11]:
dflimpio = pd.DataFrame()
dflimpio['id'] = df_test['id']
dflimpio['Price_euros'] = df_test['Price_euros']

In [12]:
dflimpio.to_csv('precios/df_test_largooo      l .csv', index=False)